In [3]:
import numpy as np
import os
import cv2
import h5py
import math
import time
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

In [4]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Activation, Input, add
import prepare_data as pd
from tensorflow.keras.optimizers import Adam, SGD

데이터 LR, HR 분리

In [8]:
# downdraded data 생성

def prepare_images(path, factor):
    
    # loop through the file in the directory 
    for file in os.listdir(path):
        
        # open the file
        img = cv2.imread(path + '/' + file)
        
        # find old and new image dimensions
        h, w, c = img.shape
        new_height = int(h / factor)
        new_width = int(w / factor)
        
        # resize the image - down
        img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
        
        # resize the image - up
        # img = cv2.resize(img, (w, h), interpolation=cv2.INTER_CUBIC)
        
        # save the image
        print('Saving {}'.format(file))
        cv2.imwrite('downgraded/{}'.format(file), img)

In [9]:
prepare_images('./train', 2)

Saving 0.png
Saving 1.png
Saving 10.png
Saving 100.png
Saving 101.png
Saving 102.png
Saving 103.png
Saving 104.png
Saving 105.png
Saving 106.png
Saving 107.png
Saving 108.png
Saving 109.png
Saving 11.png
Saving 110.png
Saving 111.png
Saving 112.png
Saving 113.png
Saving 114.png
Saving 115.png
Saving 116.png
Saving 117.png
Saving 118.png
Saving 119.png
Saving 12.png
Saving 120.png
Saving 121.png
Saving 122.png
Saving 123.png
Saving 124.png
Saving 125.png
Saving 126.png
Saving 127.png
Saving 128.png
Saving 129.png
Saving 13.png
Saving 130.png
Saving 131.png
Saving 132.png
Saving 133.png
Saving 134.png
Saving 135.png
Saving 136.png
Saving 137.png
Saving 138.png
Saving 139.png
Saving 14.png
Saving 140.png
Saving 141.png
Saving 142.png
Saving 143.png
Saving 144.png
Saving 145.png
Saving 146.png
Saving 147.png
Saving 148.png
Saving 149.png
Saving 15.png
Saving 150.png
Saving 151.png
Saving 152.png
Saving 153.png
Saving 154.png
Saving 155.png
Saving 156.png
Saving 157.png
Saving 158.png
Savin

데이터 전처리

In [7]:
x = cv2.imread('downgraded/0.png')
x.shape

(128, 128, 3)

      - train, test 가공

In [ ]:
Random_Crop = 30
Patch_size = 32
label_size = 20
conv_side = 6
factor= 2

def prepare_data(_path):
    names = os.listdir(_path)
    names = sorted(names)
    nums = names.__len__()

    data = np.zeros((nums * Random_Crop, 1, Patch_size, Patch_size), dtype=np.double)     # data = zeros(14*30, 1, 32, 32)   이미지 단일화
    label = np.zeros((nums * Random_Crop, 1, label_size, label_size), dtype=np.double)    # label = zeros(14*30, 1, 20, 20)  이미지 단일화

    for i in range(nums):
        name = _path + names[i]
        hr_img = cv2.imread(name, cv2.IMREAD_COLOR)
        shape = hr_img.shape

        hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
        hr_img = hr_img[:, :, 0]
        
        # 저화질 이미지 생성 (행렬로만 이루어져 있음)
        w, h, c = shape
        new_height = int(h / factor)
        new_width = int(w / factor)
        lr_img = cv2.resize(hr_img, (new_height, new_width))           # shape (h, w)
        lr_img = cv2.resize(lr_img, (shape[1], shape[0]))

        # produce Random_Crop random coordinate to crop training img
        Points_x = np.random.randint(0, min(shape[0], shape[1]) - Patch_size, Random_Crop)        # points_x = random(0, 낮은 값 - 32, 30)
        Points_y = np.random.randint(0, min(shape[0], shape[1]) - Patch_size, Random_Crop)        # points_y = random(0, 낮은 값 - 32, 30)
        
        # 이미지 단일화
        for j in range(Random_Crop):
            lr_patch = lr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]            # lr_patch = (n번째 + 32, n번째 + 32)
            hr_patch = hr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]

            lr_patch = lr_patch.astype(float) / 255.
            hr_patch = hr_patch.astype(float) / 255.

            data[i * Random_Crop + j, 0, :, :] = lr_patch
            label[i * Random_Crop + j, 0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]
            # cv2.imshow("lr", lr_patch)
            # cv2.imshow("hr", hr_patch)
            # cv2.waitKey(0)
    return data, label

In [ ]:
# train 데이터
train_data, train_label = prepare_data('./train/')
print(train_data.shape, train_label.shape)

# test 데이터
test_data, test_label = prepare_data('./test/')
print(test_data.shape, test_label.shape)

      - h5파일로 저장

In [ ]:
def write_hdf5(data, labels, output_filename):
    
    x = data.astype(np.float32)
    y = labels.astype(np.float32)

    with h5py.File(output_filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)
        # h.create_dataset()

In [ ]:
# train, test 데이터 저장
write_hdf5(train_data, train_label, 'train.h5')
write_hdf5(test_data, test_label, 'test.h5')

      - train, test 데이터 불러오기

In [ ]:
x_train, y_train = pd.read_training_data("train.h5")
x_val, y_val = pd.read_training_data("test.h5")

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

모델링

In [ ]:
input_img = Input(shape=IMG_SIZE)

model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(input_img)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)

model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)

model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)

model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = Activation('relu')(model)
model = Conv2D(1, (3, 3), padding='same', kernel_initializer='he_normal')(model)
res_img = model

output_img = add([res_img, input_img])

model = Model(input_img, output_img)

# SRCNN.summary()
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

훈련

In [ ]:
# 모델 훈련
start = time.time()
model.fit_generator(image_gen(train_list), steps_per_epoch=len(train_list) // BATCH_SIZE,  \
					validation_data=image_gen(test_list), validation_steps=len(train_list) // BATCH_SIZE, \
					epochs=200, workers=8)
print("걸린 시간 : ", round(end, 2))

평가

In [ ]:
# define function for peak signal-to-noise ration (PSNR)
def psnr(target, ref) : 
    
    # assume RGB/BGR image
    target_data = target.astype(float)
    ref_data = ref.astype(float)
    
    diff = ref_data - target_data
    diff = diff.flatten('C')
    
    rmse = math.sqrt(np.mean(diff ** 2.))
    
    return 20 * math.log10(255./ rmse)

# define function for mean squared error (MSE)
def mse(target, ref):
    
    # MSE is the sum of the squared fifference between the two the images
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    
    return err

# define function that combines all three image quality metrics
def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel= True))
    
    return scores

예측

In [ ]:
def modcrop(img, scale):
    tmpsz = img.shape
    sz = tmpsz[0:2]
    sz = sz - np.mod(sz, scale)
    img = img[0:sz[0], 1:sz[1]]
    return img

def shave(image, border):
    img = image[border:-border, border:-border]
    return img

In [ ]:
def predict(image_path):
    
    # load the degraded and reference images
    path, file = os.path.split(image_path)
    degraded = cv2.imread(image_path)
    ref = cv2.imread('train/{}'.format(file))
    
    # preprocess the image with madcrop
    ref = modcrop(ref, 2)
    degraded = modcrop(degraded, 2)
    
    # convert the image to VCrCb - (srccn trained on Y channel)
    temp = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
    
    # create image slice and normalize
    Y = np.zeros((1, temp.shape[0], temp.shape[1], 1), dtype=float)
    Y[0, :, :, 0] = temp[:, :, 0].astype(float) / 255
    
    # perform super-resolution with srcnn
    pre = model.predict(Y, batch_size=1)
    
    # post-process output
    pre *= 255
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(np.uint8)
    
    # copy Y channel back to image and convert to BGR
    temp = shave(temp, 6)
    temp[:, :, 0] = pre[0, :, :, 0]
    output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR) 
    
    # remove border from reference and degraded image
    ref = shave(ref.astype(np.uint8), 6)
    degraded = shave(degraded.astype(np.uint8), 6)
    
    # image quality caluclations 
    scores = []
    scores.append(compare_images(degraded, ref))
    scores.append(compare_images(output, ref))
    
    # return images and scores
    return ref, degraded, output, scores  

결과물 출력

In [ ]:
ref, degraded, output, scores = predict('downgraded/468.png')

# print all scores for all images
print('Degraded Image : \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[0][0], scores[0][1], scores[0][2]))
print('Reconstructed Image : \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[1][0], scores[1][1], scores[1][2]))

# display images as subplots
fig, axs = plt.subplots(1, 3, figsize=(20,8))
axs[0].imshow(cv2.cvtColor(ref, cv2.COLOR_BGR2RGB))
axs[0].set_title("Original")
axs[1].imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
axs[1].set_title("Degraded")
axs[2].imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
axs[2].set_title("VDSR")

# remove the x and y tick marks
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])